In [1]:
from collections.abc import Callable
import functools
import random
import re
import textwrap
from typing import Any

from huggingface_hub import InferenceApi

In [2]:
_PARA_BREAK = re.compile(r'\n{2,}')
"""Regular expression matching a paragraph break."""

'Regular expression matching a paragraph break.'

In [3]:
class CompletionError(Exception):
    """An error in attempting to complete text with BLOOM."""
    pass

In [4]:
class UnexpectedResponseError(AssertionError):
    """An unexpected error completing text. Probably a bug."""
    pass

In [5]:
class Supplier:
    """As the value of a BLOOM parameter, is called instead of used."""

    __slots__ = ('_func',)

    __match_args__ = ('_func',)

    def __init__(self, func: Callable[[], Any]) -> None:
        self._func = func
    
    def __repr__(self) -> str:
        return f'{type(self).__name__}({self._func!r})'
    
    def __call__(self) -> Any:
        return self._func()

In [6]:
@functools.cache
def _get_inference_api() -> InferenceApi:
    with open('.hf_token', encoding='utf-8') as file:
        api_token = file.read().strip()

    return InferenceApi(repo_id='bigscience/bloom', token=api_token)

In [7]:
def _normalize_paragraphs(text: str) -> str:
    """Remove any hard wrapping and separate paragraphs by single newlines."""
    pretty = _PARA_BREAK.split(text)
    raw = (para.strip().replace('\n', ' ') for para in pretty)
    return '\n'.join(raw)

In [8]:
def _prettify_paragraphs(text: str) -> str:
    """Add hard wrapping and separate paragraphs by blank lines."""
    raw = text.strip().split('\n')
    pretty = ('\n'.join(textwrap.wrap(graf)) for graf in raw)
    return '\n\n'.join(pretty)

In [9]:
class Completer:
    """High-level interface to Bloom completion."""

    __slots__ = ('_inference', '_text', '__dict__')

    def __init__(self, prompt: str, **parameters: Any) -> None:
        """Create a completer for the given prompt and optional parameters."""
        self._inference = _get_inference_api()
        self.text = _normalize_paragraphs(prompt)

        if any(name.startswith('_') for name in parameters):
            raise TypeError('cannot set parameter name with leading "_"')
        
        self._set_defaults()
        self.__dict__.update(parameters)

    def __str__(self) -> str:
        """Prettified version of the text stored so far."""
        return _prettify_paragraphs(self.text)
    
    @property
    def text(self) -> str:
        """The prompt plus any appended completions."""
        return self._text
    
    @text.setter
    def text(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError('prompt must be a string')
        if not value:
            raise ValueError('prompt must be nonempty')
        self._text = value

    def __call__(self) -> str:
        """Attempt to complete the stored text, and print all text so far."""
        self.complete()
        print(self)
    
    def complete(self) -> None:
        """Attempt to complete the stored text, but do not print anything."""
        match self._inference(inputs=self.text, params=self._build_params()):
            case [{'generated_text': completion}]:
                self._text = completion  # Whatever BLOOM returns, we accept.
            case {'error': [*errors]}:
                raise CompletionError(*errors)
            case other_response:
                raise UnexpectedResponseError(other_response)
    
    def _set_defaults(self) -> None:
        self.do_sample = True
        self.max_new_tokens = 250
        self.seed = Supplier(random.random)
        self.temperature = 0.75
    
    def _build_params(self) -> dict[str, Any]:
        return {name: (value() if isinstance(value, Supplier) else value)
                for name, value in sorted(self.__dict__.items())}


In [10]:
bloom1 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We license the
color red to you, but you haven't been paying the license fees." I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""")

In [11]:
bloom1()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We
license the color red to you, but you haven't been paying the license
fees." I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. I had to find another job. I tried making
some swords for a couple of knights, but one of them ran out of money
and the other one was starving. I tried making a tie for a senator. I
ended up with a tie that read, "burn the senator."

"No!" I said, trying to sound brave. "Oh, my! Oh, my! Oh, my! Who are
you?" I jumped up and down, my hands in the air as if I were a rabbit.

"Mom." I opened my mouth to say, "I'm going to die," but I didn't get
the words out before she was in my arms.

"You're going to be fine. Don't worry." She kissed my forehead. "Your
dad is making up for all that I missed."

We had a great time and some ama

In [12]:
bloom2 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician,
""")

In [13]:
bloom2()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician, she had a certain amount of experience fighting
off parasites. She wondered if she could summon enough willpower to
keep him attached long enough to find out just how he worked.

“Stop!” Beverly cried as the ghost’s hand brushed her cheek. She
grasped the lamp’s base and wrenched it away from him. He tumbled to
the ground, his light extinguished.

Beverly took a few steps back. She had never seen a man’s face so
transformed. Where there had been arrogant amusement there was now a
look of intense, hungry disappointment. He was done with her. He was
going to find someone else to feed off.

She thought, as soon as she saw him, just how many women would fall
for that smile. If she hadn’t been a physician, she would have fallen
completely in love with him. His fa

In [14]:
# From https://aclanthology.org/2020.acl-main.463.pdf, Appendix A.
bloom3 = Completer("""
Robinson cried out, “Help! I’m being chased by a bear! All I have is these
sticks. What should I do?” The survivalist said, “Here’s what you can do.
""")

In [15]:
bloom3()

Robinson cried out, “Help! I’m being chased by a bear! All I have is
these sticks. What should I do?” The survivalist said, “Here’s what
you can do. Grab as many sticks as you can in each hand, then run.
That will give you traction.”

When the bear caught up to Robinson, he was still carrying the two-
handed sticks. He ran into the bear and knocked him over. Then he
grabbed the bear’s paw and held it. Then he yelled, “I’m being chased
by a bear! All I have is these sticks. What should I do?”
